In [ ]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.RandomState(1234)

In [ ]:
N_pop = int(1e5)  # Population size
N_work = 100      # Number of people in a workplace
N_home = 2        # Number of people in a home

# Betas. i.e. probability of infection from a single infectious person is 
beta_work = 0.5     # Daily probability of infection if sharing workplace
beta_home = 0.6     # Daily probability of infection if sharing home
beta_street = 0.2   # Random contact on the street, transport, etc.

# Probability that an infected person recovers after i days of infection.
# These numbers are completely made up.
p_recover = np.array(([0.0] * 3) + ([0.1] * 6) + ([0.2] * 3) + ([0.3]*6) + [1.0])

In [ ]:
# Static state of the population
v_work = np.arange(N_pop) // N_work  # Index of workplace
v_home = np.arange(N_pop) // N_home  # Index of home
rng.shuffle(v_home)                  # Make workplace and home independent

In [ ]:
# Initial state of the population - no-one is infected
v_susceptible = np.full(shape=(N_pop,), fill_value=True)
v_infected = np.full(shape=(N_pop,), fill_value=False)
v_days_infected = np.full(shape=(N_pop,), fill_value=0)

In [ ]:
def stochastic_update():
    # Compute the number of infected in each workplace and home
    w_infected = np.bincount(v_work[v_infected], minlength=np.max(v_work) + 1)
    h_infected = np.bincount(v_home[v_infected], minlength=np.max(v_home) + 1)
    total_infected = np.sum(v_infected)
    
    # Compute the probability of a susceptible getting infected by various routes
    p_work = 1.0 - np.power(1.0 - beta_work / N_work, w_infected)
    p_home = 1.0 - np.power(1.0 - beta_home / N_home, h_infected)
    p_street = 1.0 - np.power(1.0 - beta_street / N_pop, total_infected)
    
    # Infection probability for individuals
    v_p = 1.0 - (1.0 - p_work[v_work]) * (1.0 - p_home[v_home]) * (1.0 - p_street)
    # Sample new infections
    new_infection = (rng.binomial(1, p=v_p) == 1)
    v_infected[v_susceptible] = new_infection[v_susceptible]
    v_susceptible[v_susceptible] = ~new_infection[v_susceptible]
    
    # Compute probability of infected individuals recovering
    v_p = p_recover[v_days_infected]
    # Sample recoveries
    new_recoveries = (rng.binomial(1, p=v_p) == 1)
    v_infected[v_infected] = ~new_recoveries[v_infected]
    
    # Increment days infected
    v_days_infected[v_infected] += 1

In [ ]:
# Randomly infect a tiny number of people
initial_infections = rng.choice(N_pop, size=4)
v_infected[initial_infections] = True

In [ ]:
susceptible_vs_t = []
infected_vs_t = []
recovered_vs_t = []

for t in range(5):
    susceptible_count = np.sum(v_susceptible)
    infected_count = np.sum(v_infected)
    recovered_count = N_pop - susceptible_count - infected_count
    susceptible_vs_t.append(susceptible_count)
    infected_vs_t.append(infected_count)
    recovered_vs_t.append(recovered_count)

    print('t = {}. Infected: {}'.format(t, np.sum(v_infected)))
    stochastic_update()